In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

In [14]:
def rsi_14_strategy(data):
    data = data.copy() 
    data['Change'] = data['Close'].diff()

    data['Gain'] = data['Change'].where(data['Change'] > 0, 0)
    data['Loss'] = -data['Change'].where(data['Change'] < 0, 0)
    
    data['Average Gain'] = np.where(data.index < data.index[14], data['Gain'].rolling(window=14).mean(), np.nan)
    data['Average Loss'] = np.where(data.index < data.index[14], data['Loss'].rolling(window=14).mean(), np.nan)

    for i in range(14, len(data)):
        data['Average Gain'].iloc[i] = (data['Average Gain'].iloc[i-1] * 13 + data['Gain'].iloc[i]) / 14
        data['Average Loss'].iloc[i] = (data['Average Loss'].iloc[i-1] * 13 + data['Loss'].iloc[i]) / 14

    data['RS'] = data['Average Gain'] / data['Average Loss']

    data['RSI'] = (100 - (100/(1+data['RS'])))
    data['RSI_action'] = np.where(data['RSI'] < 50, 1, np.where(data['RSI'] >= 50, -1,0))

    return data[['RSI','RSI_action']]


In [15]:
def ema(data, window):
    data = data.copy()
    
    first_valid_index = data['Close'].first_valid_index()
    if first_valid_index is not None:
        initial_sma_position = data.index.get_loc(first_valid_index) + window - 1
        if initial_sma_position < len(data):
            initial_sma = data['Close'].rolling(window=window).mean().iloc[initial_sma_position]
        else:
            initial_sma = np.nan
    else:
        initial_sma = np.nan

    multiplier = (2 / (window + 1))
    
    data[f'{window}ema'] = np.nan
    
    if not np.isnan(initial_sma):
        data[f'{window}ema'].iloc[initial_sma_position] = initial_sma
    
        for i in range(initial_sma_position + 1, len(data)):
            data[f'{window}ema'].iloc[i] = (data['Close'].iloc[i] - data[f'{window}ema'].iloc[i-1]) * multiplier + data[f'{window}ema'].iloc[i-1]
    
    return data[[f'{window}ema']]


In [16]:
def ema_strategy(data):
    data = data.copy()
    ema_line = ema(data,9).copy()
        
    conditions = [
        (data['Close'] > ema_line['9ema']) & (data['Close'].notna()) & (ema_line['9ema'].notna()),
        (data['Close'] < ema_line['9ema']) & (data['Close'].notna()) & (ema_line['9ema'].notna())
    ]

    choices = [1, -1]

    data['ema_strategy_action'] = np.select(conditions, choices, default=0)

    return data[['ema_strategy_action']]

In [17]:
def macd_strategy(data):
    data = data.copy()

    fastMA = ema(data, 12)
    slowMA = ema(data, 26)
    data['macd'] = fastMA['12ema'] - slowMA['26ema']

    data['macd_action'] = np.where(data['macd'] < 0, -1, np.where(data['macd'] > 0, 1,0))

    signal_data = data[['macd']].rename(columns={'macd': 'Close'})
    signal_line = ema(signal_data, 9)

    data['signal_line'] = signal_line['9ema']
    data['macd_strategy_action'] = np.where(data['macd'] > data['signal_line'], 1, np.where(data['macd'] < data['signal_line'], -1,0))
    
    return data[['macd','macd_action','signal_line','macd_strategy_action']]


In [18]:
def log_return_buy_hold(data):
    data = data.copy()
    data['log_return_buy_hold'] = np.log(data['Close'] / data['Close'].shift())

    return data[['log_return_buy_hold']]

In [33]:
def data_consolidation(tickers): #tickers in a list
    all_results = {}

    for ticker in tickers:
        data = yf.download(ticker, start='2010-01-01',)[['Close']]
        results = data.copy()

        ema_10 = ema(data, 10)
        ema_strat = ema_strategy(data)
        macd_strat = macd_strategy(data)
        rsi_strat = rsi_14_strategy(data)
        log_return_hold_strat = log_return_buy_hold(data)

        log_return_ema_strategy = ema_strat * log_return_hold_strat
        log_return_macd_strategy = macd_strat['macd_strategy_action']*log_return_hold_strat
        log_return_rsi_strategy = rsi_strat['RSI_action'] * log_return_hold_strat

        results = results.join([ema_10, ema_strat, macd_strat, rsi_strat, log_return_hold_strat, log_return_ema_strategy, log_return_macd_strategy, log_return_rsi_strategy])
        all_results[ticker] = results
        all_results_df = pd.concat(all_results, axis=1)
        all_results_df.dropna(inplace=True)
    
    return all_results_df


In [34]:
tickers = ['AAPL', 'GOOG', 'LMT', 'NFLX']
df = data_consolidation(tickers)
print(df.shape)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
(3440, 40)


In [35]:
df

AAPL                                                        \
                 Close       10ema ema_strategy_action      macd macd_action   
Date                                                                           
2010-02-22    7.157857    7.157219                  -1 -0.088343          -1   
2010-02-23    7.037857    7.135517                  -1 -0.091067          -1   
2010-02-24    7.166429    7.141137                   1 -0.081906          -1   
2010-02-25    7.214286    7.154437                   1 -0.069979          -1   
2010-02-26    7.307857    7.182332                   1 -0.052372          -1   
...                ...         ...                 ...       ...         ...   
2023-10-13  178.850006  177.564959                   1  0.106143           1   
2023-10-16  178.720001  177.774966                   1  0.214329           1   
2023-10-17  177.149994  177.661335                  -1  0.171405           1   
2023-10-18  175.839996  177.330183                  -1  0.031320           1   
2023-10-19  175.460007  176.990151                  -1 -0.109102          -1   

                                                                   \
           signal_line macd_strategy_action        RSI RSI_action   
Date                                                                
2010-02-22   -0.154489                    1  47.100030          1   
2010-02-23   -0.141805                    1  42.877325          1   
2010-02-24   -0.129825                    1  48.232536          1   
2010-02-25   -0.117856                    1  50.107469         -1   
2010-02-26   -0.104759                    1  53.642792         -1   
...                ...                  ...        ...        ...   
2023-10-13   -0.935607                    1  53.749160         -1   
2023-10-16   -0.705620                    1  53.430882         -1   
2023-10-17   -0.530215                    1  49.610274          1   
2023-10-18   -0.417908                    1  46.615111          1   
2023-10-19   -0.356147                    1  45.752238          1   

                                ...        NFLX              \
           log_return_buy_hold  ...       Close       10ema   
Date                            ...                           
2010-02-22           -0.006218  ...    9.324286    9.134704   
2010-02-23           -0.016907  ...    9.192857    9.145277   
2010-02-24            0.018104  ...    9.241429    9.162759   
2010-02-25            0.006656  ...    9.372857    9.200959   
2010-02-26            0.012887  ...    9.435714    9.243642   
...                        ...  ...         ...         ...   
2023-10-13           -0.010346  ...  355.679993  371.165289   
2023-10-16           -0.000727  ...  360.820007  369.284329   
2023-10-17           -0.008824  ...  355.720001  366.818087   
2023-10-18           -0.007422  ...  346.190002  363.067526   
2023-10-19           -0.002163  ...  401.769989  370.104338   

                                                                   \
           ema_strategy_action       macd macd_action signal_line   
Date                                                                
2010-02-22                   1   0.553660           1    0.561113   
2010-02-23                   1   0.523347           1    0.553560   
2010-02-24                   1   0.497508           1    0.542350   
2010-02-25                   1   0.482079           1    0.530296   
2010-02-26                   1   0.469511           1    0.518139   
...                        ...        ...         ...         ...   
2023-10-13                  -1 -12.656114          -1  -11.821946   
2023-10-16                  -1 -12.706830          -1  -11.998922   
2023-10-17                  -1 -13.008595          -1  -12.200857   
2023-10-18                  -1 -13.857002          -1  -12.532086   
2023-10-19                   1  -9.930063          -1  -12.011681   

                                                                           
    